In [11]:
from pyspark.sql.functions import explode, col,lit
import requests
import json

StatementMeta(, c9a51ce3-1e97-4087-a730-449767f21b04, 13, Finished, Available, Finished)

In [12]:
# Set dynamic partition overwrite mode
spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")

StatementMeta(, c9a51ce3-1e97-4087-a730-449767f21b04, 14, Finished, Available, Finished)

In [13]:
# Function to get category IDs
def get_category_ids():
    response = requests.get("https://paralleldemo.azurewebsites.net/api/LoadCategories")
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception("Failed to load categories")

StatementMeta(, c9a51ce3-1e97-4087-a730-449767f21b04, 15, Finished, Available, Finished)

In [14]:
# Function to get orders by category ID
def get_orders_by_category_id(category_id):
    url = f"https://paralleldemo.azurewebsites.net/api/LoadOrdersByCategories?categoryId={category_id}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Failed to load orders for category {category_id}")

StatementMeta(, c9a51ce3-1e97-4087-a730-449767f21b04, 16, Finished, Available, Finished)

In [15]:
# Get list of category IDs
categories = get_category_ids()

# Initialize an empty list to hold DataFrames
dfCategories=spark.createDataFrame(categories)

# Loop through each category ID and get orders
for categoryRow in dfCategories.collect():
    orders = get_orders_by_category_id(categoryRow["categoryId"])
    # Convert list of orders to DataFrame
    df = spark.createDataFrame(orders)
    # Add category_id column to DataFrame
    df = df.withColumn("CategoryId", lit(categoryRow["categoryId"]))
    df.write.partitionBy("CategoryId").mode("overwrite").saveAsTable("Orders")


StatementMeta(, c9a51ce3-1e97-4087-a730-449767f21b04, 17, Finished, Available, Finished)